In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.chdir('..')

# WritingProject

## 长文内容结构

In [2]:
from langchain_chinese import TreeContent, WritingTask
root = TreeContent()

t1 = root.add_item(TreeContent(title="H1", type="outline"))
t2 = root.add_item(TreeContent(title="H2",  type="outline"))

In [3]:
t2

'20240505.222234.00003.796'

In [4]:
h1 = root.get_item_by_id(t1)
h2_id = h1.add_item(TreeContent(text="my headline 1", title="Title1", type="outline"))
root.get_item_by_id(h2_id)

TreeContent(id='20240505.222234.00004.899', type='outline', is_completed=False, words_advice=None, title='Title1', howto=None, summarise=None, text='my headline 1', children=[], path=None)

In [5]:
root.all_todos()

[{'id': '20240505.222234.00001.992', 'type': 'outline', 'words_advice': None},
 {'id': '20240505.222234.00002.904', 'type': 'outline', 'words_advice': None},
 {'id': '20240505.222234.00004.899', 'type': 'outline', 'words_advice': None},
 {'id': '20240505.222234.00003.796', 'type': 'outline', 'words_advice': None}]

In [6]:
root.print_lines([1,6])

1.6.1 H1


1.6.1.1 Title1
my headline 1

1.6.2 H2




In [7]:
print(root.get_outlines([]))

1 H1 

1.1 Title1 

2 H2 



## 长文写作智能体

In [9]:
wp = WritingTask(task_mode="askme")
wp.run()

task_mode: askme



👤:  写1000字恐怖小说，主人工用一个具体的人名


-------------------- command: chat --------------------
-------------------- AI for 20240505.222321.00006.482 --------------------
resp: {'类型': 'outline', '标题名称': '恐怖之旅', '扩写指南': '故事围绕一名叫做李晓霞的主人公，她在一次探险中意外发现了一栋被遗弃的古老房屋。在探索房屋的秘密时，一系列诡异事件接连发生。', '总字数要求': 1000, '大纲数量': 3, '大纲列表': [{'标题名称': '意外的发现', '总字数要求': 300, '扩写指南': '描述李晓霞如何发现这栋古老房屋以及她的初步探索。'}, {'标题名称': '诡异的夜晚', '总字数要求': 400, '扩写指南': '叙述在这栋房屋中，李晓霞经历的恐怖事件以及她的心理变化。'}, {'标题名称': '真相大白', '总字数要求': 300, '扩写指南': '揭示房屋背后的秘密以及李晓霞如何逃离这个恐怖之地。'}]}
-------------------- Todos Left --------------------
20240505.222321.00006.482 | words: None



👤:  ok


-------------------- command: ok --------------------
-------------------- Outlines Done for 20240505.222321.00006.482 --------------------
-------------------- Next TODO for  20240505.222354.00007.649 --------------------
👤[auto]:  请帮我扩写《意外的发现》, 字数约为300字，扩写依据为：描述李晓霞如何发现这栋古老房屋以及她的初步探索。
-------------------- AI for 20240505.222354.00007.649 --------------------
resp: {'类型': 'paragraph', '标题名称': '意外的发现', '总字数要求': 300, '详细内容': '一次偶然的机会，李晓霞在乡间徒步时迷失了方向。在暮色四合中，她发现了一栋被茂密藤蔓包围的古老房屋。好奇心驱使下，她小心翼翼地穿过荒草丛生的庭院，推开了摇摇欲坠的大门。屋内蛛网密布，却依稀可见往日的繁华。墙上挂着的旧照片和褪色的装饰品勾起了她的兴趣。在初步探索中，李晓霞发现了通往二楼的楼梯，尽管心有畏惧，但她还是鼓起勇气，一步步踏上尘封已久的木制台阶。', '内容摘要': {'人物': '李晓霞', '地点': '古老房屋', '情节': '迷失方向，偶然发现房屋，初步探索'}}
-------------------- Todos Left --------------------
20240505.222354.00007.649 | words: 300
20240505.222354.00008.965 | words: 400
20240505.222354.00009.224 | words: 300


...

-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240505.222514.00010.423 --------------------
-------------------- Next TODO for  20240505.222514.00011.435 --------------------
👤[auto]:  请帮我扩写《心理变化》, 字数约为200字，扩写依据为：分析李晓霞在恐怖事件中的心理转变
-------------------- AI for 20240505.222514.00011.435 --------------------
resp: {'类型': 'paragraph', '标题名称': '心理变化', '总字数要求': 200, '详细内容': '起初，李晓霞对于古老房屋的诡异现象感到惊恐与不解，心中充斥着不安与惶恐。然而，随着时间的推移，她开始逐渐适应这种恐惧，内心生出一股强烈的求生欲望。在一次次恐怖事件中，李晓霞学会了观察和分析，试图寻找背后的规律和线索。她的恐惧心理逐渐转变为冷静和理智，勇敢地面对房屋中的诡异现象，寻找逃离恐怖之地的方法。', '内容摘要': {'人物': '李晓霞', '地点': '古老房屋', '情节': '李晓霞在恐怖事件中从惊恐不安到勇敢理智的心理转变'}}
-------------------- Todos Left --------------------
20240505.222514.00011.435 | words: 200
20240505.222354.00009.224 | words: 300



👤:  ok


-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240505.222514.00011.435 --------------------
-------------------- Next TODO for  20240505.222354.00009.224 --------------------
👤[auto]:  请帮我扩写《真相大白》, 字数约为300字，扩写依据为：揭示房屋背后的秘密以及李晓霞如何逃离这个恐怖之地。
-------------------- AI for 20240505.222354.00009.224 --------------------
resp: {'类型': 'paragraph', '标题名称': '真相大白', '总字数要求': 300, '详细内容': '在经历了那个诡异的夜晚后，李晓霞决定揭开古老房屋背后的秘密。她开始搜集线索，发现那些诡异现象似乎都与房屋过去的主人有关。经过一番努力，她发现原来这房屋的主人是一位被诅咒的画家，他的画作因注入了太多负面情绪而具有了诡异的生命力。画像表情的变换，窗户上的扭曲面孔，以及地板的敲击声，都是画作在夜晚释放出的恐怖气息。了解到这一真相后，李晓霞明白只有找到画家的遗作并摧毁它，才能解除诅咒。在一场紧张的搜寻中，她终于找到了那幅被诅咒的画作，并将其付之一炬。随着画作的燃烧，房屋内的诡异现象逐渐消失，李晓霞趁机逃离了这个恐怖之地。', '内容摘要': {'人物': '李晓霞', '地点': '古老房屋', '情节': '揭开房屋背后被诅咒画家的秘密，找到并摧毁画作，解除诅咒，逃离恐怖之地'}}
-------------------- Todos Left --------------------
20240505.222354.00009.224 | words: 300



👤:  ok


-------------------- command: ok --------------------
-------------------- Paragraph Done for 20240505.222354.00009.224 --------------------
-------------------- Task Complete! --------------------


In [11]:
wp.print_lines()

1 意外的发现
一次偶然的机会，李晓霞在乡间徒步时迷失了方向。在暮色四合中，她发现了一栋被茂密藤蔓包围的古老房屋。好奇心驱使下，她小心翼翼地穿过荒草丛生的庭院，推开了摇摇欲坠的大门。屋内蛛网密布，却依稀可见往日的繁华。墙上挂着的旧照片和褪色的装饰品勾起了她的兴趣。在初步探索中，李晓霞发现了通往二楼的楼梯，尽管心有畏惧，但她还是鼓起勇气，一步步踏上尘封已久的木制台阶。

2 诡异的夜晚


2.1 恐怖事件
夜幕降临，古老房屋内诡异现象逐渐浮现。墙上的画像似乎在变换表情，伴随着窗户上映出的扭曲面孔，让李晓霞不寒而栗。地板偶尔传来咚咚的敲击声，如同有某种力量在地底下挣扎。这些恐怖现象让李晓霞内心充满恐惧与不安，她开始怀疑这房屋隐藏着不为人知的秘密，而她似乎已成为其中的一部分。

2.2 心理变化
起初，李晓霞对于古老房屋的诡异现象感到惊恐与不解，心中充斥着不安与惶恐。然而，随着时间的推移，她开始逐渐适应这种恐惧，内心生出一股强烈的求生欲望。在一次次恐怖事件中，李晓霞学会了观察和分析，试图寻找背后的规律和线索。她的恐惧心理逐渐转变为冷静和理智，勇敢地面对房屋中的诡异现象，寻找逃离恐怖之地的方法。

3 真相大白
在经历了那个诡异的夜晚后，李晓霞决定揭开古老房屋背后的秘密。她开始搜集线索，发现那些诡异现象似乎都与房屋过去的主人有关。经过一番努力，她发现原来这房屋的主人是一位被诅咒的画家，他的画作因注入了太多负面情绪而具有了诡异的生命力。画像表情的变换，窗户上的扭曲面孔，以及地板的敲击声，都是画作在夜晚释放出的恐怖气息。了解到这一真相后，李晓霞明白只有找到画家的遗作并摧毁它，才能解除诅咒。在一场紧张的搜寻中，她终于找到了那幅被诅咒的画作，并将其付之一炬。随着画作的燃烧，房屋内的诡异现象逐渐消失，李晓霞趁机逃离了这个恐怖之地。



In [13]:
print(wp.root_content.get_outlines())

1 意外的发现 
{'人物': '李晓霞', '地点': '古老房屋', '情节': '迷失方向，偶然发现房屋，初步探索'}
2 诡异的夜晚 
叙述在这栋房屋中，李晓霞经历的恐怖事件以及她的心理变化。
2.1 恐怖事件 
{'人物': '李晓霞', '地点': '古老房屋', '情节': '房屋内诡异现象对李晓霞的影响，画像变换表情，窗户上的扭曲面孔，地板的敲击声，恐惧与不安'}
2.2 心理变化 
{'人物': '李晓霞', '地点': '古老房屋', '情节': '李晓霞在恐怖事件中从惊恐不安到勇敢理智的心理转变'}
3 真相大白 
{'人物': '李晓霞', '地点': '古老房屋', '情节': '揭开房屋背后被诅咒画家的秘密，找到并摧毁画作，解除诅咒，逃离恐怖之地'}


In [88]:
wp = WritingProject(streaming=True)
wp.run()


👤:  帮我写一个500字小故事


{}
{'类型': ''}
{'类型': 'out'}
{'类型': 'outlines'}
{'类型': 'outlines', '大纲列表': []}
{'类型': 'outlines', '大纲列表': [{}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和主人公'}, {'标题名称': ''}]}
{'类型': 'outlines', '大纲列表': [{'标题名称': '开篇', '字数要求': 100, '内容摘要': '描述故事背景和


👤:  quit


-------------------- quit
